In [17]:
import ray
import gym
import numpy as np
import psutil
import scipy.signal
num_cpus = psutil.cpu_count(logical=False)

ray.init(num_cpus=num_cpus)

2019-09-04 22:04:56,945	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


Exception: Perhaps you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [4]:


num_tasks = 12

@ray.remote
def f(image, random_filter):
    # Do some image processing.
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

filters = [np.random.normal(size=(4, 4)) for _ in range(num_tasks)]

# Time the code below.
def test():
    for _ in range(10):
        image = np.zeros((3000, 3000))
        image_id = ray.put(image)
        ray.get([f.remote(image_id, filters[i]) for i in range(num_tasks)])
        
%time test()

CPU times: user 1.25 s, sys: 2.43 s, total: 3.69 s
Wall time: 25.7 s


In [5]:
from multiprocessing import Pool
import numpy as np
import psutil
import scipy.signal

num_cpus = psutil.cpu_count(logical=False)

def f(args):
    image, random_filter = args
    # Do some image processing.
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

pool = Pool(num_cpus)

filters = [np.random.normal(size=(4, 4)) for _ in range(num_tasks)]

# Time the code below.

def test():
    for _ in range(10):
        image = np.zeros((3000, 3000))
        pool.map(f, zip(num_tasks * [image], filters))
%time test()

CPU times: user 8.37 s, sys: 9.47 s, total: 17.8 s
Wall time: 43.2 s


In [61]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D,Bidirectional, LSTM, Dropout
from tensorflow.keras import Model
import tensorflow as tf
class MLP(Model):
    def __init__(self, LAYER_SIZE, OUTPUT_SIZE):
        super(MLP, self).__init__()
        
        self.d1 = Dense(LAYER_SIZE, activation='relu')
        self.d2 = Dense(LAYER_SIZE, activation='relu')
        self.d3 = Dense(OUTPUT_SIZE, activation = 'tanh')

    def call(self, x, acts=None, training=False):
        
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        return x

model = MLP(12, 5)
model2 = MLP(13,4)
model(np.ones([10,12]))

<tf.Tensor: id=15455, shape=(10, 5), dtype=float64, numpy=
array([[ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811],
       [ 0.05062235,  0.14818228, -0.32560839,  0.41634638, -0.04870811]])>

In [4]:
@ray.remote
class Person:
  def __init__(self):
    pass

    
  def test(self,weight):
        return weight.shape
    
  def ez(self):
        return 5



In [5]:
actor = Person.remote()

In [7]:
ray.get(actor.test.remote(model.trainable_variables[0].numpy()))

(12, 12)

In [ ]:
ray.get(actor.test.remote(model.trainable_variables[0]))

2019-09-04 18:03:28,762	WARNING worker.py:352 -- WARNING: Falling back to serializing objects of type <enum 'VariableSynchronization'> by using pickle. This may be inefficient.
2019-09-04 18:03:28,775	WARNING worker.py:352 -- WARNING: Falling back to serializing objects of type <enum 'VariableAggregation'> by using pickle. This may be inefficient.


In [11]:
tf.compat.v1.assign(model.trainable_variables[0], np.ones((12,12)))

<tf.Variable 'UnreadVariable' shape=(12, 12) dtype=float64, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])>

In [14]:
[n.numpy() for n in model.trainable_variables]

[array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([[-1.42202652e-01,  3.47808911e-01,  9.59553275e-02,
          1.66138875e-01, -5.82863090e-03, -4.29771430e-01,
          3.76412865e-01,  1.05054466e-02,  2.65030418e-01,
          1.19525392e-01, -3.56003166e-01, -3.52443089e-01],
   

In [15]:
x = []
x.append([n.numpy() for n in model.trainable_variables])

In [20]:
model.trainable_variables[0]+2

<tf.Tensor: id=146, shape=(12, 12), dtype=float64, numpy=
array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]])>

In [56]:
model.trainable_variables

[<tf.Variable 'mlp/dense/kernel:0' shape=(12, 12) dtype=float64, numpy=
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])>,
 <tf.Variable 'mlp/dense/bias:0' shape=(12,) dtype=float64, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>,
 <tf.Variable 'mlp/dense_1/kernel:0' shape=(12, 12) dtype=float64, numpy=
 array([[-1.42202652e-01,  3.47

In [69]:
x = model.trainable_variables
y = [i+1 for i in model.trainable_variables]
z = [i-1 for i in model.trainable_variables]


x2 = model2.trainable_variables
y2 = [i+1 for i in model2.trainable_variables]
z2 = [i-1 for i in model2.trainable_variables]

In [63]:
weights = [[x,x2], [y,y2], [z,z2]]
by_models = zip(*weights)


In [67]:
weights = [['a',2], ['b',3], ['c',8]]
for q in zip(*weights):
    print(q)

('a', 'b', 'c')
(2, 3, 8)


In [60]:

[np.sum(w)/len(w) for w in zip(*[x])]

[<tf.Tensor: id=14884, shape=(12, 12), dtype=float64, numpy=
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])>,
 <tf.Tensor: id=14887, shape=(12,), dtype=float64, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>,
 <tf.Tensor: id=14890, shape=(12, 12), dtype=float64, numpy=
 array([[-1.42202652e-01,  3.47808911e-01,  9.59553275e-02,
    

In [83]:
d = {'ok':5}
d.copy()

{'ok': 5}

In [75]:
np.sum((np.expand_dims(x[0].numpy(), 0),), axis = 0)

array([[[ 0.3973526 ,  0.23372644, -0.16991816, -0.47508506,
         -0.25340132, -0.31434749,  0.36006794, -0.28112358,
          0.00555293,  0.20267567,  0.14065435,  0.04354843],
        [-0.38942335,  0.46742368,  0.46361934,  0.22871681,
          0.14772727,  0.29518396,  0.16449136,  0.10679124,
          0.0120779 , -0.06643967, -0.25144239, -0.08196039],
        [ 0.42249216, -0.25546421,  0.16359994, -0.29694118,
         -0.12504064, -0.37510743, -0.26868091, -0.35480439,
          0.10558946,  0.07322366,  0.04522533, -0.17068743],
        [-0.42908809,  0.40899685, -0.1491241 , -0.07867091,
         -0.23094486,  0.20849508, -0.21904857, -0.20920019,
          0.44698554,  0.2427562 , -0.33457385, -0.28715636],
        [ 0.49086107, -0.42783939,  0.27332596, -0.47542985,
          0.32009296, -0.24447311, -0.19043995, -0.11937856,
         -0.05637138,  0.08062986,  0.25294316,  0.13309879],
        [ 0.4787761 ,  0.30332286,  0.24253408, -0.09815372,
          0.0513895

In [57]:
l = [x,y,z]



[np.sum(w)/len(w) for w in zip(*l)]

[<tf.Tensor: id=13398, shape=(12, 12), dtype=float64, numpy=
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])>,
 <tf.Tensor: id=13403, shape=(12,), dtype=float64, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>,
 <tf.Tensor: id=13408, shape=(12, 12), dtype=float64, numpy=
 array([[-1.42202652e-01,  3.47808911e-01,  9.59553275e-02,
    

In [ ]:
import numpy as np
import tensorflow as tf
import gym
import time
import pybullet
import reach2D
from SAC import *
from train import *

SyntaxError: can't use starred expression here (cell_name, line 5)

In [ ]:
class mlp_gail_discriminator(Model):

  def __init__(self, hidden_sizes=[32,32,32], activation = 'relu'):
    super(mlp_gail_discriminator, self).__init__()
    self.mlp = mlp(list(hidden_sizes), activation, activation)
    self.prob = Dense(1, activation='sigmoid')


  def call(self, obs, acts):
    x = tf.concat([obs,acts], axis = -1)
    x = self.mlp(x)
    prob = self.prob(x)
    return prob



In [ ]:
def discriminator_train_step(batch, expert_batch,  discriminator, discriminator_optimizer):
    batch_obs, batch_acts = batch['obs1'], batch['acts']
    batch_expert_obs, batch_expert_acts = expert_batch['obs'], expert_batch['acts']
    with tf.GradientTape() as tape:
        # We'd like to maximise the log probability of the expert actions, and minmise log prob of generated actions.
        expert_probs = discriminator(batch_expert_obs,batch_expert_acts)
        # in ML, we take gradient to minimise, therefore minimise negative log probability 
        expert_loss = -tf.math.log(expert_probs)
        agent_probs = discriminator(batch_obs,batch_acts)
        # i.e, minimise -log(1-prob_generated_is_true)
        agent_loss = -(tf.math.log(1-agent_probs))
        # and thus, the reward our SAC agent gets will be -(tf.math.log(1-agent_probs)), it is trying to maximise this, 
        # discriminator is trying to mimise it.
        loss = tf.reduce_sum(expert_loss + agent_loss)
        expert_accuracy = tf.reduce_mean(tf.cast(expert_probs > 0.5, tf.float32))
        agent_accuracy  = tf.reduce_mean(tf.cast(agent_probs < 0.5, tf.float32))


        
    gradients = tape.gradient(loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))
    return loss.numpy(), expert_accuracy.numpy(), agent_accuracy.numpy()

In [ ]:
expert_obs = np.load('collected_data/expert_obs_Pendulum-v0_Hidden_32l_25000.npy')
expert_acts = np.load('collected_data/expert_actions_Pendulum-v0_Hidden_32l_25000.npy')

def sample_expert_transitions(batch_size):
    idxs = np.random.randint(0, len(expert_obs), size=batch_size)
    return {'obs':expert_obs[idxs], 'acts':expert_acts[idxs]}

In [ ]:
def gail_run(discrim_req_acc):
    ENV_NAME='Pendulum-v0'
    env_fn = lambda : gym.make(ENV_NAME)
    hid =128
    l=2
    gamma=0.999
    steps_per_epoch=5000
    seed=0
    epochs=7
    max_ep_len = 200 # for reacher, 1000 for not. 
    exp_name = ENV_NAME+'_Hidden_'+str(hid)+'l_'+str(l)
    alpha=0.2
    batch_size=100
    lr=1e-3
    start_steps=5000 
    save_freq=1
    load = False
    render = False
    polyak=0.995
    replay_size=int(1e6)
    ac_kwargs = {}
    ac_kwargs['hidden_sizes'] = [hid]*l
    discrim_req_acc = discrim_req_acc
    tf.random.set_seed(seed)
    np.random.seed(seed)
    env, test_env = env_fn(), env_fn()
    # Get Env dimensions
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]
    SAC = SAC_model(env, obs_dim, act_dim, ac_kwargs['hidden_sizes'],lr, gamma, alpha, polyak,  load, exp_name)
    # Experience buffer
    replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=replay_size)

    #Logging 
    start_time = time.time()
    train_log_dir = 'logs/' + str(discrim_req_acc)+exp_name+':'+str(start_time)
    summary_writer = tf.summary.create_file_writer(train_log_dir)

    discriminator = mlp_gail_discriminator()
    discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    def update_models(model, replay_buffer, steps, batch_size):
        agent_accuracy = 0
        # until the discriminator is trained to sufficiently distinguish correct transitions.
        print('Updating Discriminator')
        while agent_accuracy < discrim_req_acc or expert_acurracy < discrim_req_acc:
            batch = replay_buffer.sample_batch(batch_size)
            expert_batch = sample_expert_transitions(batch_size)
            _,expert_acurracy,agent_accuracy = discriminator_train_step(batch, expert_batch, discriminator, discriminator_optimizer)
            print(expert_acurracy, agent_accuracy)

        # now update SAC
        print('Updating Policy')
        for j in range(steps):
            batch = replay_buffer.sample_batch(batch_size)
            batch_obs, batch_acts = batch['obs1'], batch['acts']
            agent_probs = discriminator(batch_obs,batch_acts)
            agent_reward = -(tf.math.log(1-agent_probs)).numpy().squeeze().astype('float32')
            # use GAIL reward instead of environment reward
            batch['rews'] = agent_reward

            LossPi, LossQ1, LossQ2, LossV, Q1Vals, Q2Vals, VVals, LogPi = model.train_step(batch)


    # now collect epsiodes
    total_steps = steps_per_epoch * epochs
    steps_collected = 0

    # collect some initial random steps to initialise
    random_steps = 5000
    steps_collected  += rollout_trajectories(n_steps = random_steps,env = env, max_ep_len = max_ep_len, actor = 'random', replay_buffer = replay_buffer, summary_writer = summary_writer)


    update_models(SAC, replay_buffer, steps = random_steps, batch_size = batch_size)

    # now act with our actor, and alternately collect data, then train.
    while steps_collected < total_steps:
    # collect an episode
        steps_collected  += rollout_trajectories(n_steps = max_ep_len,env = env, max_ep_len = max_ep_len, actor = SAC.get_action, replay_buffer = replay_buffer, summary_writer=summary_writer, current_total_steps = steps_collected)
        # take than many training steps
        update_models(SAC, replay_buffer, steps = max_ep_len, batch_size = batch_size)

        # if an epoch has elapsed, save and test.
        if steps_collected  > 0 and steps_collected  % steps_per_epoch == 0:
            #SAC.save_weights()
            # Test the performance of the deterministic version of the agent.
            rollout_trajectories(n_steps = max_ep_len*10,env = test_env, max_ep_len = max_ep_len, actor = SAC.get_deterministic_action, summary_writer=summary_writer, current_total_steps = steps_collected, train = False, render = True)


In [ ]:
test_accs = [0.6, 0.7, 0.8, 0.9]
for a in test_accs:
    gail_run(a)

In [6]:
d = {'ok':5, 'cu':78}
for k,v in d.items():
    print(k,v)

ok 5
cu 78


In [9]:
for i, (k,v) in enumerate(d.items()):
    print(i,k,v)

0 ok 5
1 cu 78


In [ ]:
# Okay, we need to do two ablations.
# Whats the optimal accuracy that we want our discriminator to have before we train? It'll be different on every problem
# but lets see if changing it on this simple problem can give us some ideas of what will happen.

# How 

In [ ]:
batch_size = 100
expert_batch = sample_expert_transitions(batch_size)
batch_expert_obs, batch_expert_acts = expert_batch['obs'], expert_batch['acts']
expert_probs = discriminator(batch_expert_obs,batch_expert_acts)


agent_reward = -(tf.math.log(1-expert_probs)).numpy().squeeze().astype('float32')
agent_reward

In [ ]:
discriminator = mlp_gail_discriminator()


In [ ]:
-(tf.math.log(0.0)).numpy()

In [ ]:
-(tf.math.log(1-0.01)).numpy()

In [ ]:
import matplotlib.pyplot as plt
xs = np.linspace(0,1,100)

In [ ]:
def f(x):
    return -(tf.math.log(1-x+1e-8)).numpy()

In [ ]:
ys = [f(x) for x in xs]

In [ ]:
plt.plot(xs,ys)

In [ ]:
def f2(x):
    return (tf.math.log(x+1e-8)-(tf.math.log(1-x+1e-8)).numpy())
ys = [f2(x) for x in xs]
plt.plot(xs,ys)

In [ ]:
def f3(x):
    return tf.math.log(x+1e-8).numpy()
ys = [f3(x) for x in xs]
plt.plot(xs,ys)

In [ ]:
x = np.linspace(1,100,100)


In [ ]:
np.delete(x, [1,2,3])

In [ ]:
a = tf.ones((100,1))
a = a.numpy().squeeze()

In [18]:
ray.shutdown()

In [ ]:
q[.shapeq

In [ ]:
a = np.array([11,21,31])

In [ ]:
b = np.array([1,2])

In [ ]:
a[b]

In [ ]:
5 % 1000

In [ ]:
tf.tensor([np.ones(100)])

In [ ]:
env

In [ ]:
env.action_space

In [ ]:
env = gym.make('Pendulum-v0')

In [ ]:
env.reset()

In [ ]:
env.step(1)

In [ ]:
a